## Configuration and Setup

In [12]:
import os
import shutil
import csv
import glob
import time
import json
import math
import random
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Optional

import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import amp
import torch.multiprocessing as mp
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import FileLink, display

# set reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.benchmark = True

# set multiprocessing start method
try:
    mp.set_start_method('spawn', force=True)
except RuntimeError:
    pass

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# creating necessary directories
Path("/kaggle/working/models").mkdir(parents=True, exist_ok=True)
Path("/kaggle/working/logs").mkdir(parents=True, exist_ok=True)

Using device: cuda


## Dataset Manager Class

In [2]:
class DatasetManager:
    def __init__(self, source_root: Path, working_root: Path):
        self.source_root = source_root
        self.working_root = working_root
        self.merge_map = {
            "ringworm": "fungal_infection",
            "demodicosis": "mange", 
            "hypersensitivity": "dermatitis",
            "flea_allergy": "dermatitis",
            "hotspot": "dermatitis",
            "skin_lesion": None
        }
        self.splits = ["train", "valid", "test"]
        self.retired_root = working_root.parent / "_retired_classes"
        
    def count_files(self, directory: Path) -> int:
        return sum(1 for p in directory.rglob("*") if p.is_file())
    
    def scan_dataset(self, root: Path) -> Dict:
        counts = {}
        for split in self.splits:
            split_dir = root / split
            if not split_dir.exists():
                counts[split] = {}
                continue
                
            split_counts = {}
            for class_dir in sorted([p for p in split_dir.iterdir() if p.is_dir()]):
                split_counts[class_dir.name] = self.count_files(class_dir)
            counts[split] = split_counts
        return counts
    
    def validate_merge_map(self, dataset_counts: Dict) -> bool:
        valid = True
        all_classes = set()
        
        for split_counts in dataset_counts.values():
            all_classes.update(split_counts.keys())
        
        for src_class, dst_class in self.merge_map.items():
            if dst_class and dst_class not in all_classes:
                print(f"Target class '{dst_class}' not found in dataset")
                valid = False
                
        return valid
    
    def merge_dataset(self, dry_run: bool = False) -> Dict:
        if not self.working_root.exists():
            print(f"Copying dataset from {self.source_root} to {self.working_root}")
            shutil.copytree(self.source_root, self.working_root)
        
        print("=== Before Merge ===")
        before_counts = self.scan_dataset(self.working_root)
        for split in self.splits:
            total = sum(before_counts[split].values())
            print(f"{split}: {total} files")
            for cls, count in sorted(before_counts[split].items()):
                print(f"  {cls:>20s}: {count}")
        
        if not self.validate_merge_map(before_counts):
            print("Merge map validation failed!")
            return before_counts
        
        log_file = Path("/kaggle/working/logs/merge_log.csv")
        log_file.parent.mkdir(exist_ok=True)
        
        total_moved = 0
        with open(log_file, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["split", "src_path", "dst_path", "src_class", "dst_class", "timestamp"])
            
            for split in self.splits:
                split_dir = self.working_root / split
                if not split_dir.exists():
                    continue
                    
                for src_class, dst_class in self.merge_map.items():
                    src_dir = split_dir / src_class
                    if not src_dir.exists():
                        continue
                        
                    if dst_class is None:
                        dst_dir = self.retired_root / split / src_class
                        action = "RETIRE"
                    else:
                        dst_dir = split_dir / dst_class
                        action = f"MERGE→{dst_class}"
                    
                    print(f"[{split}] {src_class} -> {dst_class or 'retired'} ({action})")
                    
                    dst_dir.mkdir(parents=True, exist_ok=True)
                    
                    moved = 0
                    for file_path in src_dir.rglob("*"):
                        if not file_path.is_file():
                            continue
                            
                        rel_path = file_path.relative_to(src_dir)
                        target_path = dst_dir / rel_path
                        
                        if target_path.exists():
                            stem, suffix = target_path.stem, target_path.suffix
                            i = 1
                            while (dst_dir / f"{stem}__m{i}{suffix}").exists():
                                i += 1
                            target_path = dst_dir / f"{stem}__m{i}{suffix}"
                        
                        if not dry_run:
                            shutil.move(str(file_path), str(target_path))
                        
                        writer.writerow([split, str(file_path), str(target_path), 
                                       src_class, dst_class or "retired", time.time()])
                        moved += 1
                        total_moved += 1
                    
                    if not dry_run:
                        for p in sorted(src_dir.rglob("*"), reverse=True):
                            if p.is_dir() and not any(p.iterdir()):
                                p.rmdir()
                        if src_dir.exists() and not any(src_dir.iterdir()):
                            src_dir.rmdir()
        
        print("\n=== After Merge ===")
        after_counts = self.scan_dataset(self.working_root)
        final_classes = set()
        
        for split in self.splits:
            total = sum(after_counts[split].values())
            print(f"{split}: {total} files")
            for cls, count in sorted(after_counts[split].items()):
                print(f"  {cls:>20s}: {count}")
                final_classes.add(cls)
        
        class_info = {
            'classes': sorted(final_classes),
            'class_to_idx': {cls: i for i, cls in enumerate(sorted(final_classes))},
            'idx_to_class': {i: cls for i, cls in enumerate(sorted(final_classes))},
            'merge_map': self.merge_map,
            'counts': after_counts
        }
        
        with open("/kaggle/working/class_info.json", "w") as f:
            json.dump(class_info, f, indent=2)
        
        print(f"\nTotal files moved: {total_moved}")
        print(f"Final classes: {sorted(final_classes)}")
        print(f"Log saved to: {log_file}")
        print(f"Class info saved to: /kaggle/working/class_info.json")
        
        return after_counts

# executing dataset merging
source_root = Path("/kaggle/input/imagedataset")
working_root = Path("/kaggle/working/imagedataset_merged")
dataset_manager = DatasetManager(source_root, working_root)
final_counts = dataset_manager.merge_dataset(dry_run=False)

=== Before Merge ===
train: 19264 files
   bacterial_infection: 1750
            dermatitis: 3219
           eye_redness: 1716
                 fleas: 195
      fungal_infection: 3926
               healthy: 4717
             keratosis: 1155
                 mange: 1365
       nasal_discharge: 1221
valid: 3825 files
   bacterial_infection: 305
            dermatitis: 579
           eye_redness: 439
                 fleas: 12
      fungal_infection: 755
               healthy: 547
             keratosis: 383
                 mange: 402
       nasal_discharge: 403
test: 1752 files
   bacterial_infection: 63
            dermatitis: 163
           eye_redness: 352
                 fleas: 18
      fungal_infection: 290
               healthy: 297
             keratosis: 195
                 mange: 180
       nasal_discharge: 194

=== After Merge ===
train: 19264 files
   bacterial_infection: 1750
            dermatitis: 3219
           eye_redness: 1716
                 fleas: 195
      fun

In [3]:
def check_data_leakage():
    train_files = {f.name for f in Path("/kaggle/working/imagedataset_merged/train").rglob('*') if f.is_file()}
    val_files = {f.name for f in Path("/kaggle/working/imagedataset_merged/valid").rglob('*') if f.is_file()}
    test_files = {f.name for f in Path("/kaggle/working/imagedataset_merged/test").rglob('*') if f.is_file()}
    
    leak_train_val = train_files.intersection(val_files)
    leak_train_test = train_files.intersection(test_files)
    leak_val_test = val_files.intersection(test_files)
    
    print(f"Train-Val leakage: {len(leak_train_val)} files")
    print(f"Train-Test leakage: {len(leak_train_test)} files")
    print(f"Val-Test leakage: {len(leak_val_test)} files")
    
    if leak_train_val or leak_train_test or leak_val_test:
        print("Data leakage detected!")
        return True
    else:
        print("No data leakage detected.")
        return False

has_leakage = check_data_leakage()

Train-Val leakage: 0 files
Train-Test leakage: 0 files
Val-Test leakage: 0 files
No data leakage detected.


In [4]:
# loading class information from JSON file
with open("/kaggle/working/class_info.json", "r") as f:
    class_info = json.load(f)

# extracting class information for model configuration
CLASS_NAMES = class_info["classes"]
NUM_CLASSES = len(CLASS_NAMES)
CLASS_TO_IDX = class_info["class_to_idx"]
IDX_TO_CLASS = class_info["idx_to_class"]

print(f"Classes: {CLASS_NAMES}")
print(f"Number of classes: {NUM_CLASSES}")

# setting image and training parameters
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_WORKERS = 4
PIN_MEMORY = torch.cuda.is_available()

# defining data augmentation transforms for training
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# defining validation transforms (no augmentation)
val_transforms = transforms.Compose([
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# splitting the datasets for train, validation, and test 
train_dataset = datasets.ImageFolder(Path("/kaggle/working/imagedataset_merged/train"), transform=train_transforms)
val_dataset = datasets.ImageFolder(Path("/kaggle/working/imagedataset_merged/valid"), transform=val_transforms)
test_dataset = datasets.ImageFolder(Path("/kaggle/working/imagedataset_merged/test"), transform=val_transforms)

assert train_dataset.classes == CLASS_NAMES, "Class mismatch!"

# calculating class weights for handling imbalanced data
train_targets = np.array([label for _, label in train_dataset.samples])
class_counts = np.bincount(train_targets, minlength=NUM_CLASSES)
beta = 0.999
effective_num = 1.0 - np.power(beta, class_counts)
class_weights = effective_num.sum() / (effective_num + 1e-12)
class_weights = class_weights / class_weights.mean()
class_weights = torch.tensor(class_weights, dtype=torch.float32, device=DEVICE)

print("Class counts:", class_counts)
print("Class weights:", class_weights.cpu().numpy())

# creating data loaders for training and evaluation
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

Classes: ['bacterial_infection', 'dermatitis', 'eye_redness', 'fleas', 'fungal_infection', 'healthy', 'keratosis', 'mange', 'nasal_discharge']
Number of classes: 9
Class counts: [1750 3219 1716  195 3926 4717 1155 1365 1221]
Class weights: [0.7089593  0.6102356  0.7141511  3.3054044  0.59763205 0.59114194
 0.855127   0.7866184  0.8307303 ]


## Defining The Model

In [5]:
# custom EfficientNet model with regularization
class RegularizedEfficientNet(nn.Module):
    def __init__(self, num_classes: int, dropout_rate: float = 0.4):
        super().__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)
        
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        # replacing classifier head with custom layers
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier[1] = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )
        
    def forward(self, x):
        return self.base_model(x)
    
    def unfreeze_layers(self, num_layers: int = 10):
        for param in self.base_model.classifier.parameters():
            param.requires_grad = True
            
        total_layers = len(list(self.base_model.features))
        layers_to_unfreeze = min(num_layers, total_layers)
        
        for i, (name, param) in enumerate(self.base_model.features.named_parameters()):
            if i >= total_layers - layers_to_unfreeze:
                param.requires_grad = True

# initializing model and move to device
model = RegularizedEfficientNet(num_classes=NUM_CLASSES, dropout_rate=0.4)
model.to(DEVICE)

# defining loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
# Using AdamW optimizer for regularization
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3, eps=1e-8)
scaler = torch.cuda.amp.GradScaler()

# printing model parameter count
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

Model parameters: 4,019,077


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_172/56632578.py:39: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


## Creating the Medical Response System

In [6]:
class MedicalResponseSystem:
    def __init__(self):
        # treatment protocols for different medical conditions
        self.treatment_protocols = {
            'bacterial_infection': {
                'treatments': ['Topical antibiotics (mupirocin)', 'Oral antibiotics (cephalexin)', 'Wound cleaning', 'Warm compresses'],
                'emergency': False,
                'urgency': 'moderate'
            },
            'dermatitis': {
                'treatments': ['Topical corticosteroids', 'Moisturizers', 'Antihistamines', 'Avoid triggers'],
                'emergency': False, 
                'urgency': 'low'
            },
            'eye_redness': {
                'treatments': ['Artificial tears', 'Cool compresses', 'Antibiotic eye drops (if bacterial)', 'Avoid rubbing'],
                'emergency': True,
                'urgency': 'high'
            },
            'fleas': {
                'treatments': ['Flea control on pet', 'Topical anti-itch creams', 'Oral antihistamines', 'Environmental cleaning'],
                'emergency': False,
                'urgency': 'low'
            },
            'fungal_infection': {
                'treatments': ['Antifungal creams (clotrimazole)', 'Oral antifungals (terbinafine)', 'Keep area dry', 'Antifungal powders'],
                'emergency': False,
                'urgency': 'moderate'
            },
            'healthy': {
                'treatments': ['Maintain skin hygiene', 'Regular moisturizing', 'Sun protection'],
                'emergency': False,
                'urgency': 'none'
            },
            'keratosis': {
                'treatments': ['Moisturizers with urea/lactic acid', 'Gentle exfoliation', 'Topical retinoids', 'Laser therapy for severe cases'],
                'emergency': False,
                'urgency': 'low'
            },
            'mange': {
                'treatments': ['Veterinary-prescribed acaricides', 'Medicated dips', 'Oral medications', 'Environmental decontamination'],
                'emergency': True,
                'urgency': 'high'
            },
            'nasal_discharge': {
                'treatments': ['Saline nasal spray', 'Decongestants', 'Antihistamines if allergic', 'Humidifier'],
                'emergency': False,
                'urgency': 'moderate'
            }
        }
        
        self.emergency_indicators = {
            'difficulty_breathing': 'EMERGENCY: Seek immediate medical care',
            'swollen_face': 'URGENT: Could indicate allergic reaction',
            'high_fever': 'URGENT: May indicate systemic infection', 
            'rapid_spread': 'URGENT: Condition worsening quickly',
            'vision_changes': 'EMERGENCY: Eye symptoms affecting vision',
            'lethargy': 'URGENT: Systemic involvement possible'
        }
    
    def get_treatment_plan(self, disease_name, confidence_score):
        # getting treatment protocol for specific disease
        protocol = self.treatment_protocols.get(disease_name, {
            'treatments': ['Consult healthcare professional for proper diagnosis'],
            'emergency': False,
            'urgency': 'unknown'
        })

        # returning comprehensive treatment plan
        return {
            'diagnosis': disease_name,
            'treatments': protocol['treatments'],
            'is_emergency': protocol['emergency'],
            'urgency_level': protocol['urgency'],
            'confidence': confidence_score,
            'recommendation': self._generate_recommendation(protocol, confidence_score)
        }
    
    def _generate_recommendation(self, protocol, confidence):
        # generating recommendation based on urgency and confidence
        if protocol['emergency']:
            return "EMERGENCY: Seek immediate veterinary/medical attention"
        
        if confidence < 0.7:
            return "Low confidence diagnosis - Consult healthcare professional"
        
        if protocol['urgency'] == 'high':
            return "URGENT: Consult healthcare professional within 24 hours"
        elif protocol['urgency'] == 'moderate':
            return "Consult healthcare professional within 2-3 days"
        else:
            return "Monitor condition - consider over-the-counter options"
    
    def assess_emergency_symptoms(self, symptoms_text):
        emergency_flags = []
        if symptoms_text:
            symptoms_lower = symptoms_text.lower()
            for indicator, message in self.emergency_indicators.items():
                if indicator in symptoms_lower:
                    emergency_flags.append(message)
        
        return emergency_flags
    
    def generate_comprehensive_report(self, image_prediction, symptoms_text=None):
        # generating complete medical report combining image prediction and symptoms
        primary_diagnosis = image_prediction[0]
        
        report = {
            'primary_diagnosis': primary_diagnosis['class'],
            'confidence': primary_diagnosis['confidence'],
            'differential_diagnoses': image_prediction[1:3],
            'medical_advice': self.get_treatment_plan(primary_diagnosis['class'], primary_diagnosis['confidence'])
        }
        
        if symptoms_text:
            report['emergency_assessment'] = self.assess_emergency_symptoms(symptoms_text)
            report['patient_symptoms'] = symptoms_text
        
        report['disclaimer'] = "AI-assisted diagnosis. Consult healthcare professional for medical decisions."
        
        return report

# initializing medical response system
medical_system = MedicalResponseSystem()

## Model Trainer Class

In [7]:
class ModelTrainer:
    def __init__(self, model, criterion, optimizer, device, scaler=None):
        # initializing trainer with model, loss function and optimizer
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.scaler = scaler
        self.history = []  
        
    def train_epoch(self, train_loader, epoch):
        self.model.train()
        total_loss, correct, total = 0, 0, 0
        
        # progress bar for training
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
        for batch_idx, (data, target) in enumerate(pbar):
            data, target = data.to(self.device), target.to(self.device)
            
            self.optimizer.zero_grad()
            
            with amp.autocast('cuda'):
                output = self.model(data)
                loss = self.criterion(output, target)
            
            # backward pass with gradient scaling
            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)
            
            pbar.set_postfix({
                'loss': f'{total_loss/(batch_idx+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
        
        return total_loss / len(train_loader), 100. * correct / total
    
    def validate(self, val_loader, desc="[Val]"):
        # validating model performance
        self.model.eval()
        total_loss, correct, total = 0, 0, 0
        all_preds, all_targets = [], []  
        
        with torch.no_grad():
            pbar = tqdm(val_loader, desc=desc)
            for data, target in pbar:
                data, target = data.to(self.device), target.to(self.device)
                
                with amp.autocast('cuda'):
                    output = self.model(data)
                    loss = self.criterion(output, target)
                
                total_loss += loss.item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
                total += target.size(0)
                
                all_preds.extend(pred.cpu().numpy())
                all_targets.extend(target.cpu().numpy())
                
                pbar.set_postfix({
                    'loss': f'{total_loss/len(val_loader):.4f}',
                    'acc': f'{100.*correct/total:.2f}%'
                })
        
        return total_loss / len(val_loader), 100. * correct / total, all_preds, all_targets
    
    def save_checkpoint(self, path, epoch, is_best=False):
        # save model checkpoint with training state
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'history': self.history,
            'class_info': {
                'classes': CLASS_NAMES,
                'class_to_idx': CLASS_TO_IDX,
                'idx_to_class': IDX_TO_CLASS
            }
        }
        torch.save(checkpoint, path)
        # save an additional copy of best model 
        if is_best:
            best_path = path.replace('.pth', '_best.pth')
            torch.save(checkpoint, best_path)

# initializing model trainer
trainer = ModelTrainer(model, criterion, optimizer, DEVICE, scaler)

## K-Fold Cross Validation

In [8]:
def create_k_fold_datasets(n_splits=3): 
    # creating k-fold splits using indices
    all_indices = list(range(len(train_dataset)))
    all_labels = [label for _, label in train_dataset.samples]
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    folds = []
    
    for train_idx, val_idx in skf.split(all_indices, all_labels):
        fold = {
            'train_indices': train_idx.tolist(),
            'val_indices': val_idx.tolist(),
        }
        folds.append(fold)
    
    return folds

# creating dataset subset using specific indices
class SubsetDataset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

# creating k-fold splits
folds = create_k_fold_datasets(n_splits=3)
print(f"Created {len(folds)} stratified folds")

Created 3 stratified folds


In [9]:
def train_k_fold_model(folds, num_epochs=8):
    fold_results = []
    
    for fold_idx, fold_data in enumerate(folds):
        print(f"\n=== Training Fold {fold_idx + 1}/{len(folds)} ===")
        
        # using SubsetDataset with the indices 
        train_subset = SubsetDataset(train_dataset, fold_data['train_indices'])
        val_subset = SubsetDataset(train_dataset, fold_data['val_indices'])
        
        train_loader = DataLoader(
            train_subset, batch_size=BATCH_SIZE, shuffle=True,
            num_workers=0, pin_memory=False  
        )
        val_loader = DataLoader(
            val_subset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=0, pin_memory=False  
        )
        
        # creating model with updated API
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        
        # replacing classifier
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(in_features, NUM_CLASSES)
        )
        
        # freezing initially
        for param in model.parameters():
            param.requires_grad = False
        for param in model.classifier.parameters():
            param.requires_grad = True
            
        model.to(DEVICE)
        
        # using updated GradScaler API
        optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
        total_steps = num_epochs * len(train_loader)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
        scaler = torch.amp.GradScaler(device='cuda')
        
        best_val_acc, fold_history = 0, []
        
        for epoch in range(num_epochs):
            # gradual unfreezing
            if epoch == 1:  
                total_layers = len(list(model.features))
                for i, (name, param) in enumerate(model.features.named_parameters()):
                    if i >= total_layers - 3: 
                        param.requires_grad = True
            
            model.train()
            train_loss, train_acc = 0, 0
            
            pbar = tqdm(train_loader, desc=f"Fold {fold_idx+1} Epoch {epoch+1}")
            for batch_idx, (data, target) in enumerate(pbar):
                data, target = data.to(DEVICE), target.to(DEVICE)
                
                optimizer.zero_grad()
                with amp.autocast('cuda'):
                    output = model(data)
                    loss = criterion(output, target)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                train_loss += loss.item()
                pred = output.argmax(dim=1)
                train_acc += (pred == target).float().mean().item()
                
                # updating progress bar 
                if batch_idx % 10 == 0:
                    pbar.set_postfix({
                        'loss': f'{train_loss/(batch_idx+1):.4f}',
                        'acc': f'{train_acc/(batch_idx+1):.3f}'
                    })
            
            # validation
            model.eval()
            val_loss, val_acc = 0, 0
            with torch.no_grad():
                for data, target in val_loader:
                    data, target = data.to(DEVICE), target.to(DEVICE)
                    with amp.autocast('cuda'):
                        output = model(data)
                        loss = criterion(output, target)
                    val_loss += loss.item()
                    pred = output.argmax(dim=1)
                    val_acc += (pred == target).float().mean().item()
            
            # calculating averages
            train_loss /= len(train_loader)
            train_acc /= len(train_loader)
            val_loss /= len(val_loader)
            val_acc /= len(val_loader)
            
            fold_history.append({
                'epoch': epoch + 1,
                'train_loss': train_loss,
                'train_acc': train_acc,
                'val_loss': val_loss,
                'val_acc': val_acc
            })
            
            print(f"Epoch {epoch+1}: Train Acc: {train_acc:.3f}, Val Acc: {val_acc:.3f}")
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"/kaggle/working/models/fold_{fold_idx}_best.pth")
        
        fold_results.append({
            'fold_idx': fold_idx,
            'best_val_acc': best_val_acc,
            'history': fold_history
        })
    
    return fold_results

# running the model
fold_results = train_k_fold_model(folds, num_epochs=8)


=== Training Fold 1/3 ===


Fold 1 Epoch 1:   0%|          | 0/402 [00:00<?, ?it/s]

Epoch 1: Train Acc: 0.511, Val Acc: 0.650


Fold 1 Epoch 2:   0%|          | 0/402 [00:00<?, ?it/s]

Epoch 2: Train Acc: 0.777, Val Acc: 0.878


...

Epoch 6: Train Acc: 0.931, Val Acc: 0.932


Fold 3 Epoch 7:   0%|          | 0/402 [00:00<?, ?it/s]

Epoch 7: Train Acc: 0.938, Val Acc: 0.931


Fold 3 Epoch 8:   0%|          | 0/402 [00:00<?, ?it/s]

Epoch 8: Train Acc: 0.944, Val Acc: 0.941


## Final Model Training & Performance Analysis

In [10]:
# initializing final model with regularization
final_model = RegularizedEfficientNet(num_classes=NUM_CLASSES, dropout_rate=0.4)
final_model.to(DEVICE)

# setting up optimizer, scheduler, and mixed precision training
final_optimizer = torch.optim.AdamW(final_model.parameters(), lr=3e-4, weight_decay=1e-3)
final_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(final_optimizer, T_max=13 * len(train_loader))
final_scaler = torch.cuda.amp.GradScaler()

# initializing model trainer
final_trainer = ModelTrainer(final_model, criterion, final_optimizer, DEVICE, final_scaler)

# early stopping configuration
best_val_acc, patience, epochs_no_improve = 0, 2, 0

# training loop
for epoch in range(13):
    train_loss, train_acc = final_trainer.train_epoch(train_loader, epoch)
    val_loss, val_acc, _, _ = final_trainer.validate(val_loader)
    
    final_trainer.history.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc
    })
    
    # saving best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        final_trainer.save_checkpoint(f"/kaggle/working/models/final_checkpoint_epoch_{epoch+1}.pth", epoch, is_best=True)
        print(f"New best model: {val_acc:.2f}%")
    else:
        epochs_no_improve += 1
        
    if epochs_no_improve >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

# saving final model checkpoint
final_trainer.save_checkpoint("/kaggle/working/models/final_model.pth", 19)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_172/1585204824.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  final_scaler = torch.cuda.amp.GradScaler()


Epoch 1 [Train]:   0%|          | 0/602 [00:11<?, ?it/s]

[Val]:   0%|          | 0/120 [00:11<?, ?it/s]

New best model: 62.98%


Epoch 2 [Train]:   0%|          | 0/602 [00:11<?, ?it/s]

...

Epoch 8 [Train]:   0%|          | 0/602 [00:11<?, ?it/s]

[Val]:   0%|          | 0/120 [00:11<?, ?it/s]

Epoch 9 [Train]:   0%|          | 0/602 [00:11<?, ?it/s]

[Val]:   0%|          | 0/120 [00:11<?, ?it/s]

Early stopping at epoch 9


In [13]:
fold_accuracies = [result['best_val_acc'] for result in fold_results]
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)

print(f"\nK-Fold Cross-Validation Results:")
print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")
print(f"Individual Fold Accuracies: {[f'{acc:.3f}' for acc in fold_accuracies]}")

# loading the checkpoint and extracting just model weights
best_model_files = glob.glob("/kaggle/working/models/final_checkpoint_epoch_*_best.pth")
if best_model_files:
    best_model_path = best_model_files[0]
    print(f"Loading best model: {best_model_path}")
    
    checkpoint = torch.load(best_model_path)
    final_model.load_state_dict(checkpoint['model_state_dict'])
    
else:
    print("Loading final model (best not found)")
    final_model.load_state_dict(torch.load("/kaggle/working/models/final_model.pth"))

# testing the model
test_loss, test_acc, test_preds, test_targets = final_trainer.validate(test_loader, desc="[Final Test]")

print(f"\nFinal Test Results:")
print(f"Accuracy: {test_acc:.2f}%")
print(f"K-Fold Mean: {mean_accuracy:.3f} ± {std_accuracy:.3f}")


K-Fold Cross-Validation Results:
Mean Accuracy: 0.941 ± 0.002
Individual Fold Accuracies: ['0.944', '0.940', '0.941']
Loading best model: /kaggle/working/models/final_checkpoint_epoch_7_best.pth


[Final Test]:   0%|          | 0/55 [00:11<?, ?it/s]


Final Test Results:
Accuracy: 73.23%
K-Fold Mean: 0.941 ± 0.002


## Skin Disease Predictor

In [14]:
class SkinDiseasePredictor:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # loading class info from checkpoint
        self.class_names = CLASS_NAMES.copy()
        self.class_to_idx = CLASS_TO_IDX.copy()
        self.idx_to_class = IDX_TO_CLASS.copy()
        self.num_classes = len(self.class_names)
        
        # loading weights and update class info from checkpoint
        try:
            best_model_files = glob.glob("/kaggle/working/models/final_checkpoint_epoch_*_best.pth")
            model_files = glob.glob("/kaggle/working/models/*.pth")
            
            # loading best model
            if best_model_files:
                model_path = best_model_files[0]
            elif model_files:
                model_path = model_files[0]
            else:
                print("No model weights found: using initialized model")
                # Create model with current classes
                self.model = RegularizedEfficientNet(num_classes=self.num_classes, dropout_rate=0.4)
                self.model.to(self.device)
                self.model.eval()
                return
                
            print(f"Loading model from: {model_path}")
            checkpoint = torch.load(model_path, map_location=self.device)
            
            # checking if the checkpoint has class info that differs from current
            if 'class_info' in checkpoint:
                saved_class_info = checkpoint['class_info']
                print(f"Saved class info found: {saved_class_info['classes']}")
                print(f"Current class info: {self.class_names}")
                
                # updating mappings if they differ
                if saved_class_info['classes'] != self.class_names:
                    print("Class mismatch detected: Using saved class mappings.")
                    self.class_names = saved_class_info['classes']
                    self.class_to_idx = saved_class_info['class_to_idx']
                    self.idx_to_class = saved_class_info['idx_to_class']
                    self.num_classes = len(self.class_names)
            
            # handling different checkpoint formats
            if 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
            elif 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
            else:
                state_dict = checkpoint
            
            # creating model with the correct number of classes
            self.model = RegularizedEfficientNet(num_classes=self.num_classes, dropout_rate=0.4)
            self.model.to(self.device)
            
            self.model.load_state_dict(state_dict)
            self.model.eval()
            
            print(f"Loaded model weights from {model_path}")
            print(f"Model configured for {self.num_classes} classes: {self.class_names}")
                
        except Exception as e:
            print(f"Could not load weights: {e}. Using initialized model.")
            self.model = RegularizedEfficientNet(num_classes=self.num_classes, dropout_rate=0.4)
            self.model.to(self.device)
            self.model.eval()
        
        self.transform = val_transforms
        self.medical_system = MedicalResponseSystem()

    def predict(self, image_path, top_k=3):
        try:
            # opening and preprocessing image
            image = Image.open(image_path).convert('RGB')
            input_tensor = self.transform(image).unsqueeze(0).to(self.device)
            
            with torch.no_grad():
                output = self.model(input_tensor)
                probabilities = F.softmax(output, dim=1)
            
            # getting top predictions
            top_k = min(top_k, self.num_classes)
            top_probs, top_indices = torch.topk(probabilities, top_k)
            top_probs = top_probs.cpu().numpy()[0]
            top_indices = top_indices.cpu().numpy()[0]
            
            predictions = []
            for i, (idx, prob) in enumerate(zip(top_indices, top_probs)):
                class_name = self.idx_to_class.get(str(idx), f"unknown_class_{idx}")
                predictions.append({
                    'rank': i + 1,
                    'class': class_name,
                    'confidence': float(prob),
                    'class_index': int(idx)
                })
            
            return predictions
            
        except Exception as e:
            print(f"Prediction error: {e}")
            import traceback
            traceback.print_exc()
            return [{
                'rank': 1,
                'class': 'prediction_error',
                'confidence': 0.0,
                'error': str(e)
            }]
    
    def predict_with_treatment(self, image_path, symptoms_text=None):
        image_pred = self.predict(image_path)
        
        # checking if prediction failed
        if image_pred and 'error' in image_pred[0]:
            return {
                'error': image_pred[0]['error'],
                'diagnosis': 'unknown',
                'recommendation': 'Please try again or use a different image'
            }
        
        report = self.medical_system.generate_comprehensive_report(image_pred, symptoms_text)
        return report

# initializing predictor
predictor = SkinDiseasePredictor()
print("The Predictor has been initialized successfully!")

# testing with a real image
test_images = list(Path("/kaggle/working/imagedataset_merged/test").rglob("*.jpg"))
if test_images:
    test_image_path = str(test_images[0])
    print(f"\nTesting with: {test_image_path}")
    
    # getting prediction
    predictions = predictor.predict(test_image_path)
    print("\nPREDICTIONS:")
    for pred in predictions:
        if 'error' in pred:
            print(f"Error: {pred['error']}")
        else:
            print(f"{pred['rank']}. {pred['class']}: {pred['confidence']:.3f}")
    
    # getting full medical report if prediction was successful
    if not any('error' in pred for pred in predictions):
        medical_report = predictor.predict_with_treatment(
            test_image_path, 
            "red itchy skin with some swelling"
        )
        
        print("\nMedical Report:")
        print(json.dumps(medical_report, indent=2))
    else:
        print("Skipping medical report due to prediction error")
else:
    print("No test images found")

Loading model from: /kaggle/working/models/final_checkpoint_epoch_7_best.pth
Saved class info found: ['bacterial_infection', 'dermatitis', 'eye_redness', 'fleas', 'fungal_infection', 'healthy', 'keratosis', 'mange', 'nasal_discharge']
Current class info: ['bacterial_infection', 'dermatitis', 'eye_redness', 'fleas', 'fungal_infection', 'healthy', 'keratosis', 'mange', 'nasal_discharge']
Loaded model weights from /kaggle/working/models/final_checkpoint_epoch_7_best.pth
Model configured for 9 classes: ['bacterial_infection', 'dermatitis', 'eye_redness', 'fleas', 'fungal_infection', 'healthy', 'keratosis', 'mange', 'nasal_discharge']
The Predictor has been initialized successfully!

Testing with: /kaggle/working/imagedataset_merged/test/eye_redness/eye_redness_e7efa92d57.jpg

PREDICTIONS:
1. dermatitis: 0.360
2. fleas: 0.191
3. fungal_infection: 0.177

Medical Report:
{
  "primary_diagnosis": "dermatitis",
  "confidence": 0.36026519536972046,
  "differential_diagnoses": [
    {
      "rank

## Saving the Model

In [15]:
# creating directory for all model assets
os.makedirs('model_assets', exist_ok=True)

# saving the trained model weights
best_model_files = glob.glob("/kaggle/working/models/final_checkpoint_epoch_*_best.pth")
if best_model_files:
    checkpoint = torch.load(best_model_files[0])
    torch.save(checkpoint['model_state_dict'], 'model_assets/skin_disease_model.pth')
else:
    torch.save(model.state_dict(), 'model_assets/skin_disease_model.pth')

# saving class mappings and names
with open('model_assets/class_to_idx.json', 'w') as f:
    json.dump(CLASS_TO_IDX, f, indent=2)

with open('model_assets/idx_to_class.json', 'w') as f:
    json.dump(IDX_TO_CLASS, f, indent=2)

with open('model_assets/class_names.json', 'w') as f:
    json.dump(CLASS_NAMES, f, indent=2)

# saving medical response system configuration
with open('model_assets/medical_protocols.json', 'w') as f:
    json.dump(medical_system.treatment_protocols, f, indent=2)

with open('model_assets/emergency_indicators.json', 'w') as f:
    json.dump(medical_system.emergency_indicators, f, indent=2)

# saving model configuration
model_config = {
    "model_name": "EfficientNet-B0",
    "num_classes": NUM_CLASSES,
    "input_size": 224,
    "normalization": {
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225]
    },
    "device": str(DEVICE)
}

with open('model_assets/model_config.json', 'w') as f:
    json.dump(model_config, f, indent=2)

In [16]:
# saving the MedicalResponseSystem class code
medical_system_code = '''
class MedicalResponseSystem:
    def __init__(self, protocols_path, emergency_indicators_path):
        # Load from saved files instead of hardcoded
        with open(protocols_path, 'r') as f:
            self.treatment_protocols = json.load(f)
        
        with open(emergency_indicators_path, 'r') as f:
            self.emergency_indicators = json.load(f)
    
    def get_treatment_plan(self, disease_name, confidence_score):
        """Your existing treatment plan logic"""
        protocol = self.treatment_protocols.get(disease_name, {
            'treatments': ['Consult healthcare professional for proper diagnosis'],
            'emergency': False,
            'urgency': 'unknown'
        })
        
        return {
            'diagnosis': disease_name,
            'treatments': protocol['treatments'],
            'is_emergency': protocol['emergency'],
            'urgency_level': protocol['urgency'],
            'confidence': confidence_score,
            'recommendation': self._generate_recommendation(protocol, confidence_score)
        }
    
    def _generate_recommendation(self, protocol, confidence):
        """Your existing recommendation logic"""
        if protocol['emergency']:
            return "EMERGENCY: Seek immediate veterinary/medical attention"
        
        if confidence < 0.7:
            return "Low confidence diagnosis - Consult healthcare professional"
        
        if protocol['urgency'] == 'high':
            return "URGENT: Consult healthcare professional within 24 hours"
        elif protocol['urgency'] == 'moderate':
            return "Consult healthcare professional within 2-3 days"
        else:
            return "Monitor condition - consider over-the-counter options"
    
    def assess_emergency_symptoms(self, symptoms_text):
        """Your existing emergency assessment logic"""
        emergency_flags = []
        if symptoms_text:
            symptoms_lower = symptoms_text.lower()
            for indicator, message in self.emergency_indicators.items():
                if indicator in symptoms_lower:
                    emergency_flags.append(message)
        
        return emergency_flags
    
    def generate_comprehensive_report(self, image_prediction, symptoms_text=None):
        """Your existing report generation logic"""
        primary_diagnosis = image_prediction[0]
        
        report = {
            'primary_diagnosis': primary_diagnosis['class'],
            'confidence': primary_diagnosis['confidence'],
            'differential_diagnoses': image_prediction[1:3],
            'medical_advice': self.get_treatment_plan(primary_diagnosis['class'], primary_diagnosis['confidence'])
        }
        
        if symptoms_text:
            report['emergency_assessment'] = self.assess_emergency_symptoms(symptoms_text)
            report['patient_symptoms'] = symptoms_text
        
        report['disclaimer'] = "AI-assisted diagnosis. Consult healthcare professional for medical decisions."
        
        return report
'''

# creating the inference script that uses the class itself
inference_script = f'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import json
import io
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np

# the MedicalResponseSystem class 
{medical_system_code}

# the Skin Disease Predictor Class 
class SkinDiseasePredictor:
    def __init__(self, model_assets_path):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_assets_path = model_assets_path
        
        # loading configuration
        with open(f'{{model_assets_path}}/model_config.json', 'r') as f:
            self.config = json.load(f)
        
        # loading class mappings
        with open(f'{{model_assets_path}}/class_to_idx.json', 'r') as f:
            self.class_to_idx = json.load(f)
        
        with open(f'{{model_assets_path}}/idx_to_class.json', 'r') as f:
            self.idx_to_class = json.load(f)
        
        # loading class names
        with open(f'{{model_assets_path}}/class_names.json', 'r') as f:
            self.class_names = json.load(f)
        
        # initializing the MedicalResponseSystem
        self.medical_system = MedicalResponseSystem(
            f'{{model_assets_path}}/medical_protocols.json',
            f'{{model_assets_path}}/emergency_indicators.json'
        )
        
        # initializing model
        self.model = self._create_model()
        self.model.load_state_dict(
            torch.load(f'{{model_assets_path}}/skin_disease_model.pth', 
                      map_location=self.device)
        )
        self.model.to(self.device)
        self.model.eval()
        
        # image transforms
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(self.config["input_size"]),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=self.config["normalization"]["mean"],
                std=self.config["normalization"]["std"]
            ),
        ])
    
    def _create_model(self):
        class RegularizedEfficientNet(nn.Module):
            def __init__(self, num_classes=9, dropout_rate=0.4):
                super().__init__()
                self.base_model = models.efficientnet_b0(pretrained=False)
                in_features = self.base_model.classifier[1].in_features
                self.base_model.classifier[1] = nn.Sequential(
                    nn.Dropout(dropout_rate),
                    nn.Linear(in_features, num_classes)
                )
            
            def forward(self, x):
                return self.base_model(x)
                
        return RegularizedEfficientNet(self.config["num_classes"])
    
    def predict(self, image_bytes, top_k=3):
        # predicting diseases from image
        try:
            image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
            input_tensor = self.transform(image).unsqueeze(0).to(self.device)
            
            with torch.no_grad():
                output = self.model(input_tensor)
                probabilities = F.softmax(output, dim=1)
                top_probs, top_indices = torch.topk(probabilities, top_k)
            
            top_probs = top_probs.cpu().numpy()[0]
            top_indices = top_indices.cpu().numpy()[0]
            
            predictions = []
            for i, (idx, prob) in enumerate(zip(top_indices, top_probs)):
                class_name = self.idx_to_class[str(idx)]
                predictions.append({{
                    'rank': i + 1,
                    'class': class_name,
                    'confidence': float(prob),
                    'class_index': int(idx)
                }})
            
            return predictions
            
        except Exception as e:
            return [{{
                'rank': 1,
                'class': 'prediction_error',
                'confidence': 0.0,
                'error': str(e)
            }}]
    
    def predict_with_treatment(self, image_bytes, symptoms_text=None):
        image_pred = self.predict(image_bytes)
        
        # checking if prediction failed
        if image_pred and 'error' in image_pred[0]:
            return {{
                'error': image_pred[0]['error'],
                'diagnosis': 'unknown',
                'recommendation': 'Please try again or use a different image'
            }}
        
        # generating report
        report = self.medical_system.generate_comprehensive_report(image_pred, symptoms_text)
        return report

# flask app
app = Flask(__name__)
CORS(app)

# initializing predictor
predictor = None

def initialize_predictor():
    global predictor
    if predictor is None:
        predictor = SkinDiseasePredictor('model_assets')
    return predictor

@app.route('/predict', methods=['POST'])
def predict_endpoint():
    if 'image' not in request.files:
        return jsonify({{'error': 'No image file provided'}}), 400
    
    file = request.files['image']
    if file.filename == '':
        return jsonify({{'error': 'No file selected'}}), 400
    
    try:
        predictor = initialize_predictor()
        image_bytes = file.read()
        symptoms = request.form.get('symptoms', '')
        
        # Use your existing medical response system
        result = predictor.predict_with_treatment(image_bytes, symptoms)
        return jsonify(result)
    
    except Exception as e:
        return jsonify({{'error': str(e)}}), 500

@app.route('/health', methods=['GET'])
def health_check():
    predictor = initialize_predictor()
    return jsonify({{
        'status': 'healthy', 
        'model_loaded': True,
        'num_classes': len(predictor.class_names),
        'classes': predictor.class_names
    }})

if __name__ == '__main__':
    initialize_predictor()
    print("Skin Disease Prediction API started!")
    print(f"Loaded {{len(predictor.class_names)}} classes: {{predictor.class_names}}")
    app.run(host='0.0.0.0', port=5000, debug=True)
'''

with open('model_assets/inference.py', 'w') as f:
    f.write(inference_script)

In [17]:
# creating requirements.txt
requirements = '''
torch>=1.9.0
torchvision>=0.10.0
pillow>=8.0.0
numpy>=1.21.0
flask>=2.0.0
flask-cors>=3.0.0
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

In [18]:
# creating zip archive of model assets
shutil.make_archive('/kaggle/working/model_assets', 'zip', '/kaggle/working/model_assets')
print("Created complete model_assets.zip archive")

# creating zip of essential files
essential_files = [
    'requirements.txt',
    'model_assets/inference.py',
    'model_assets/skin_disease_model.pth',
    'model_assets/class_to_idx.json',
    'model_assets/idx_to_class.json',
    'model_assets/class_names.json',
    'model_assets/medical_protocols.json',
    'model_assets/emergency_indicators.json',
    'model_assets/model_config.json'
]

os.makedirs('/kaggle/working/essential_files', exist_ok=True)
for file_path in essential_files:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, f'/kaggle/working/essential_files/{file_name}')

shutil.make_archive('/kaggle/working/essential_files', 'zip', '/kaggle/working/essential_files')

Created complete model_assets.zip archive


'/kaggle/working/essential_files.zip'